In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Kidney_Disease_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\Kidney_Disease_Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass (frozen= True)
class TrainingConfig:
    root_dir :Path
    trained_model_path: Path
    updated_model_path: Path
    training_data: Path
    params_epoch: int
    params_batch_size: int
    params_is_augamentation: bool
    params_image_size:list

In [6]:
from src.Kidney_Disease_Classifier.constants import *
from src.Kidney_Disease_Classifier.utils.common import read_yaml,create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_data,"kidney-ct-scan-image" )


        create_directories([training.root_dir])


        training_config = TrainingConfig(
            root_dir = training.root_dir,
            trained_model_path = training.trained_model_path,
            updated_model_path = prepare_base_model.updated_model_path,
            training_data = Path(training_data),
            params_epoch = params.EPOCHS,
            params_batch_size = params.BATCH_SIZE,
            params_is_augamentation = params.AUGUMRENTATION,
            params_image_size = params.IMAGE_SIZE
        )

        return training_config

In [8]:
import os
import tensorflow as tf


In [9]:
class Training:
    def __init__(self, config = TrainingConfig):
        self.config = config

    def get_base_model(self):
        """ This method responsible to load model from updated model from base model pipeline"""
        self.model = tf.keras.models.load_model(
            self.config.updated_model_path
        )

    
    def train_valid_generator(self):

        data_generator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.2
            )
        
        dataflow_kwargs = dict(
            target_size = self.config.params_image_size[:-1],
            interpolation = "bilinear",
            batch_size =self.config.params_batch_size

        )
        
        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
        )

        self.valid_generator = valid_data_generator.flow_from_directory(
            directory=self.config.training_data,
            subset= "validation",
            shuffle=False,
            **dataflow_kwargs,
        )

        if self.config.params_is_augamentation:
            training_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **data_generator_kwargs
            )

        else:
            training_datagenerator = valid_data_generator

        self.train_generator = training_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='training',
            shuffle= False,
            **dataflow_kwargs
        )
    @staticmethod
    def save_model(path : Path, model:tf.keras.Model):
        model.save(path)

    def train(self):
        self.step_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size


        self.model.fit(
            self.train_generator,
            epochs = self.config.params_epoch,
            steps_per_epoch = self.step_per_epoch,
            validation_steps = self.validation_steps,
            validation_data = self.valid_generator
        )

        self.save_model(
            path= self.config.trained_model_path,
            model = self.model
        )

In [12]:
try:
    config = ConfigurationManager()
    trainer_config = config.get_model_trainer()
    training = Training(config= trainer_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()

except Exception as e:
    raise e

[2023-11-13 12:57:33,842 - INFO - common - Reading data is completed form config\config.yaml]
[2023-11-13 12:57:33,864 - INFO - common - Reading data is completed form params.yaml]
[2023-11-13 12:57:33,867 - INFO - common - Directory is created for artifacts]
[2023-11-13 12:57:33,871 - INFO - common - Directory is created for artifacts/model_trainer]


Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
Epoch 1/10
23/23 [==============================] - 92s 4s/step - loss: 28.8335 - accuracy: 0.4755 - val_loss: 46.5845 - val_accuracy: 0.4375
Epoch 2/10
23/23 [==============================] - 89s 4s/step - loss: 13.5845 - accuracy: 0.6657 - val_loss: 40.6110 - val_accuracy: 0.4375
Epoch 3/10
23/23 [==============================] - 83s 4s/step - loss: 18.6950 - accuracy: 0.6039 - val_loss: 32.1403 - val_accuracy: 0.4375
Epoch 4/10
23/23 [==============================] - 80s 3s/step - loss: 20.5147 - accuracy: 0.6348 - val_loss: 28.8134 - val_accuracy: 0.4375
Epoch 5/10
23/23 [==============================] - 80s 3s/step - loss: 15.4443 - accuracy: 0.6433 - val_loss: 3.0044 - val_accuracy: 0.8875
Epoch 6/10
23/23 [==============================] - 174s 8s/step - loss: 12.3707 - accuracy: 0.7051 - val_loss: 15.1438 - val_accuracy: 0.6000
Epoch 7/10
23/23 [==============================] - 83s 4s/step - l